# Hyperelastic Model Fitting

In [1]:
%load_ext autoreload
%autoreload 2
%matmodlab bokeh
from pandas import read_excel

BokehJS successfully loaded.

Populating the interactive namespace from matmodlab and bokeh


## Experimental Data

In [2]:
# uniaxial data
udf = read_excel('Treloar_hyperelastic_data.xlsx', sheetname='Uniaxial')
ud = udf.as_matrix(columns=('Engineering Strain', 'Engineering Stress (MPa)'))

# Biaxial data
bdf = read_excel('Treloar_hyperelastic_data.xlsx', sheetname='Biaxial')
bd = bdf.as_matrix(columns=('Engineering Strain', 'Engineering Stress (MPa)'))

# Pure shear data
sdf = read_excel('Treloar_hyperelastic_data.xlsx', sheetname='Pure Shear')
sd = sdf.as_matrix(columns=('Engineering Strain', 'Engineering Stress (MPa)'))

## Uniaxial Data

Find the optimal fit to the uniaxial stress data with a hyperelastic polynomial model.  The symbolic constant `UNIAXIAL_DATA` instructs `hyperopt` to interpret the input data as coming from a uniaxial stress experiment.

In [3]:
uf = hyperopt(UNIAXIAL_DATA, ud[:,0], ud[:,1])
print uf.summary()

            Data type: Uniaxial
Number of data points: 23
     Polynomial order: 5
        I2 dependence: True
           Parameters: C10=-395859933.045, C01=132250968.904, C20=-134170727.507, C11=104493666.738, C02=-33928679.075, C30=11734697.899, C21=-11610156.849, C12=-24806492.415, C03=17653757.438, C40=-4337674.748, C31=-4054457.364, C22=-1034831.948, C13=3875358.131, C04=9768482.730, C50=-270732.550, C41=901202.455, C32=-1893102.941, C23=989459.036, C14=4834111.469, C05=-3146197.619
                Error: 0.0198322188292
        


At this point, the optimal parameters have been determined and are accessible with the `popt` attribute:

In [4]:
uf.popt

array([ -3.95859933e+08,   1.32250969e+08,  -1.34170728e+08,
         1.04493667e+08,  -3.39286791e+07,   1.17346979e+07,
        -1.16101568e+07,  -2.48064924e+07,   1.76537574e+07,
        -4.33767475e+06,  -4.05445736e+06,  -1.03483195e+06,
         3.87535813e+06,   9.76848273e+06,  -2.70732550e+05,
         9.01202455e+05,  -1.89310294e+06,   9.89459036e+05,
         4.83411147e+06,  -3.14619762e+06])

The optimal parameters are also available as a dictionary via the `todict` method:

In [5]:
uf.todict()

{'C01': 132250968.90355907,
 'C02': -33928679.074741118,
 'C03': 17653757.438015793,
 'C04': 9768482.730201792,
 'C05': -3146197.6186751584,
 'C10': -395859933.04529548,
 'C11': 104493666.73756613,
 'C12': -24806492.414594918,
 'C13': 3875358.1313264915,
 'C14': 4834111.4689746331,
 'C20': -134170727.5069174,
 'C21': -11610156.84929673,
 'C22': -1034831.9476936288,
 'C23': 989459.03600392921,
 'C30': 11734697.899416655,
 'C31': -4054457.3641825169,
 'C32': -1893102.9406949782,
 'C40': -4337674.7481137849,
 'C41': 901202.45488339872,
 'C50': -270732.54991535202}

The error in the fit:

In [6]:
uf.error

Plots are generated with the `bp_plot` method

In [7]:
show(uf.bp_plot())

## Biaxial Data

Biaxial data is fit in a similar manner:

In [8]:
bf = hyperopt(BIAXIAL_DATA, bd[:,0], bd[:,1])
print bf.summary()
show(bf.bp_plot())

            Data type: Biaxial
Number of data points: 16
     Polynomial order: 4
        I2 dependence: True
           Parameters: C10=48126616.023, C01=-14323537.793, C20=597202131.473, C11=-420483882.811, C02=108970996.079, C30=416916020.562, C21=-85123697.713, C12=-35925284.204, C03=-3410748.604, C40=105722602.577, C31=9671542.604, C22=2942836.173, C13=-267685.707, C04=16785.785
                Error: 0.00990862004187
        


## Shear Data

Lastly, the shear data is fit

In [9]:
sf = hyperopt(SHEAR_DATA, sd[:,0], sd[:,1])
print sf.summary()
show(sf.bp_plot())

            Data type: Shear
Number of data points: 13
     Polynomial order: 3
        I2 dependence: True
           Parameters: C10=-480403082.071, C01=161106124.736, C20=-278863992.929, C11=38177864.867, C02=56699970.680, C30=-29401178.157, C21=-4432.913, C12=2952118.412, C03=-752519.876
                Error: 0.0182429252418
        


## Comparison of Fits

Examine the error in the shear fit using parameters from the uniaxial fit

In [10]:
y1 = sf.eval(overlay=uf)
y2 = sf.eval()
err = sqrt(mean((y1-y2)**2)) / average(abs(y2))
print err
show(sf.bp_plot(overlay=[bf, uf]))
show(uf.bp_plot(overlay=[sf]))
show(bf.bp_plot(overlay=[uf, sf]))

0.0831722323787


## hyperopt2

`hyperopt2` attempts to find the model that fits all given data the best.

In [11]:
f = hyperopt2(SHEAR_DATA, sd[:,0], sd[:,1], 
              UNIAXIAL_DATA, ud[:,0], ud[:,1],
              BIAXIAL_DATA, bd[:,0], bd[:,1])

In [12]:
print f.summary()

            Data type: Uniaxial
Number of data points: 23
     Polynomial order: 4
        I2 dependence: False
           Parameters: C10=430761.746, C20=46252.207, C30=-22262.415, C40=3276.462
                Error: 0.0441582062147
        


In [13]:
f.error2

In [14]:
p = f.bp_plot(strain=linspace(0,6.5), points=False)
p.circle(sd[:,0], sd[:,1], color='black', legend='Shear data')
p.circle(bd[:,0], bd[:,1], color='red', legend='Biaxial data')
p.circle(ud[:,0], ud[:,1], color='green', legend='Uniaxial data')
show(p)